In [1]:
!pip install gdown
!gdown --id 1-aQCjHTb8X-C_iyDSvnJlAVlPQFrQa6R --output train.zip
#https://drive.google.com/file/d/1-aQCjHTb8X-C_iyDSvnJlAVlPQFrQa6R/view?usp=drive_link

/opt/conda/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-aQCjHTb8X-C_iyDSvnJlAVlPQFrQa6R
From (redirected): https://drive.google.com/uc?id=1-aQCjHTb8X-C_iyDSvnJlAVlPQFrQa6R&confirm=t&uuid=21eb71db-f209-4177-b195-aef2ad366420
To: /train.zip
100%|████████████████████████████████████████| 354M/354M [00:07<00:00, 47.3MB/s]


In [2]:
!pip install datasets gdown
!pip install torch einops
!pip install 'numpy<2'
!pip install -U sentence-transformers
!pip install 'accelerate>=0.26.0'
!pip install -q pyvi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 89.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 81.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 49.0 MB/s eta 0:00:0000:010:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 45.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 61.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 81.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [3]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformerTrainer
from sentence_transformers import losses
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers import datasets

In [4]:
from torch.utils.data import DataLoader
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.datasets import NoDuplicatesDataLoader
from sklearn.model_selection import train_test_split
from pyvi.ViTokenizer import tokenize


from datasets import Dataset
import pandas as pd
import ast

In [5]:
final = pd.read_csv("test_cross_rand_3.csv")
print(len(final))
final.head()

109456


,question,context,cid,qid,negative_cid,negative_context
0,Người học ngành quản_lý khai_thác công_trình t...,"['Khả_năng học_tập , nâng cao trình_độ - Khối_...",[62492],161615,"[35066, 134143, 65675, 242472, 188807, 147732,...",['Nâng bậc lương 1 . Người_quản_lý công_ty chu...
1,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,['Nội_dung lồng_ghép vấn_đề bình_đẳng giới tro...,[151154],80037,"[461115, 214112, 591097, 26114, 151153, 53897,...",['Khoản 3.1 - Nâng cao nhận_thức về công_tác p...
2,Sản_phẩm phần_mềm có được hưởng ưu_đãi về thời...,"[' Điều 20 . Ưu_đãi về thời_gian miễn thuế , g...",[75071],124074,"[557258, 157135, 17873, 73603, 506982, 631699,...",['Điều 7 . Tiêu_chí cụ_thể đối_với sản_phẩm ph...
3,Điều_kiện để giáo_viên trong cơ_sở giáo_dục mầ...,"['Điều_kiện được hưởng Cán_bộ quản_lý , giáo_v...",[225897],146841,"[70683, 14311, 521812, 442161, 552188, 613651,...","[' I . PHẠM_VI , ĐỐI_TƯỢNG VÀ ĐIỀU_KIỆN ÁP_DỤN..."
4,Nguyên_tắc áp_dụng phụ_cấp ưu_đãi nghề y_tế th...,['Nguyên_tắc áp_dụng 1 . Trường_hợp công_chức ...,[68365],6176,"[172835, 449105, 34366, 207806, 126900, 126677...",['Về thực_hiện chính_sách tiền_lương ... Từ ng...


In [6]:
from tqdm import tqdm
columns_to_convert = ["context", "cid", "negative_cid", "negative_context"]

for col in columns_to_convert:
    final[col] = final[col].apply(ast.literal_eval)
final.head()
# final['context'] = final['context'].str.strip("[]'")  
# final['context'] = final['context'].str.strip('"')

,question,context,cid,qid,negative_cid,negative_context
0,Người học ngành quản_lý khai_thác công_trình t...,"[Khả_năng học_tập , nâng cao trình_độ - Khối_l...",[62492],161615,"[35066, 134143, 65675, 242472, 188807, 147732,...",[Nâng bậc lương 1 . Người_quản_lý công_ty chuy...
1,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,[Nội_dung lồng_ghép vấn_đề bình_đẳng giới tron...,[151154],80037,"[461115, 214112, 591097, 26114, 151153, 53897,...",[Khoản 3.1 - Nâng cao nhận_thức về công_tác ph...
2,Sản_phẩm phần_mềm có được hưởng ưu_đãi về thời...,"[ Điều 20 . Ưu_đãi về thời_gian miễn thuế , gi...",[75071],124074,"[557258, 157135, 17873, 73603, 506982, 631699,...",[Điều 7 . Tiêu_chí cụ_thể đối_với sản_phẩm phầ...
3,Điều_kiện để giáo_viên trong cơ_sở giáo_dục mầ...,"[Điều_kiện được hưởng Cán_bộ quản_lý , giáo_vi...",[225897],146841,"[70683, 14311, 521812, 442161, 552188, 613651,...","[ I . PHẠM_VI , ĐỐI_TƯỢNG VÀ ĐIỀU_KIỆN ÁP_DỤNG..."
4,Nguyên_tắc áp_dụng phụ_cấp ưu_đãi nghề y_tế th...,"[Nguyên_tắc áp_dụng 1 . Trường_hợp công_chức ,...",[68365],6176,"[172835, 449105, 34366, 207806, 126900, 126677...",[Về thực_hiện chính_sách tiền_lương ... Từ ngà...


In [7]:
train_data, eval_data = train_test_split(final, test_size=0.1, random_state=42)
print(f"Train size: {len(train_data)}, Eval size: {len(eval_data)}")

Train size: 98510, Eval size: 10946


In [8]:
from sentence_transformers.readers import InputExample
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator


In [9]:
#multi loss
def create_contrastive_dataset(filtered_data):
    train_samples = []

    for index, row in filtered_data.iterrows():
        query = row['question']
        i = 0

        for doc_positive in row['context']:
            train_samples.append(InputExample(
                    texts=[query, doc_positive],
                    label=1  # positive pair
                ))

        for doc_negative in row['negative_context']:
            if i < 3: 
                train_samples.append(InputExample(
                    texts=[query, doc_negative],
                    label=0 # negative pair
                ))
                i+=1

    return train_samples

train_dataset = create_contrastive_dataset(train_data)
print(len(train_dataset))
validation_dataset = create_contrastive_dataset(eval_data)
print(len(validation_dataset))


405676
45026


In [15]:
from sentence_transformers import SentenceTransformer,CrossEncoder, SentenceTransformerTrainer, losses, SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from transformers import AdamW
import torch
from torch import nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CrossEncoder('itdainb/PhoRanker', max_length=256, num_labels=1)
model.model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [16]:
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
evaluator = CEBinaryClassificationEvaluator.from_input_examples(validation_dataset, name="Quora-dev")
#loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([10.0])).to(device)
loss_fn = nn.BCEWithLogitsLoss().to(device)


In [17]:
from transformers import logging
logging.set_verbosity_error()  # Tắt toàn bộ log lỗi và cảnh báo từ Transformers

In [18]:
# batch 32 - 34k step
model.fit(
    train_dataloader=train_dataloader,
    evaluator=evaluator,
    epochs=2,
    loss_fct = loss_fn,
    warmup_steps=int(len(train_dataset) * 0.1),
    optimizer_params={'lr': 3e-4, 'weight_decay': 0.01},
    #evaluation_steps=9000, #- 34k - 32batch
    evaluation_steps=1000, #- 12k
    save_best_model=True,
    output_path='models/PhoRanker_3_rand_2',
    show_progress_bar=True
)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/12678 [00:00<?, ?it/s]

Iteration:   0%|          | 0/12678 [00:00<?, ?it/s]

In [19]:
!pip install gdown
!gdown --id 1YUqZ1tIK6IRFeFFSNS3Ovv1031dl7fKZ --output test.zip

/opt/conda/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1YUqZ1tIK6IRFeFFSNS3Ovv1031dl7fKZ
From (redirected): https://drive.google.com/uc?id=1YUqZ1tIK6IRFeFFSNS3Ovv1031dl7fKZ&confirm=t&uuid=c2cb30fd-42b7-40e4-a6a1-2db701031404
To: /test.zip
100%|██████████████████████████████████████| 32.5M/32.5M [00:02<00:00, 15.1MB/s]


In [20]:
test = pd.read_csv("test_cross_rand.csv")
columns_to_convert = ["context", "cid", "negative_cid", "negative_context"]

for col in columns_to_convert:
    test[col] = test[col].apply(ast.literal_eval)
test.head()

,question,context,cid,qid,negative_cid,negative_context
0,Phó Tổng_Giám_đốc Ngân_hàng Chính_sách_xã_hội ...,[Áp_dụng chế_độ tiền_lương và phụ_cấp quy_định...,[140864],70867,"[528765, 572673, 514251, 583125, 17301, 618705...","[Điều 8 . Mức tiền_lương bình_quân kế_hoạch , ..."
1,Ai có thẩm_quyền quyết_định thành_lập Hội_đồng...,[Thành_lập Hội_đồng 1 . Bộ_trưởng Bộ Y_tế ra ...,[62339],813,"[550254, 217782, 485071, 74532, 626731, 30585,...","[Điều 12 . Hội_đồng tư_vấn , Hội_đồng quản_lý ..."
2,Thời_hiệu xử_phạt đối_với nhà_xuất_bản thực_hi...,[Điều 5 . Thời_hiệu xử_phạt vi_phạm hành_chín...,[63171],40392,"[169898, 67492, 566524, 217447, 551061, 472409...",[Thời_hiệu xử_lý vi_phạm hành_chính 1 . Thời_h...
3,"Việc ký_kết , thực_hiện thoả_thuận quốc_tế nhâ...",[Báo_cáo tình_hình ký_kết và thực_hiện thoả_th...,[157761],85946,"[218667, 168253, 511073, 510475, 565009, 63166...","[Trách_nhiệm của đơn_vị tổ_chức hội_nghị , hội..."
4,Đề_án sử_dụng tài_sản công tại đơn_vị sự_nghiệ...,[ Điều 44 . Đề_án sử_dụng tài_sản công tại đơ...,[95397],55607,"[62810, 448562, 74995, 505955, 505959, 76871, ...",[Sử_dụng tài_sản công tại đơn_vị sự_nghiệp côn...


In [21]:
from sentence_transformers import CrossEncoder
model_encode = CrossEncoder("models/PhoRanker_3_rand_2", max_length=256, num_labels=1)

In [65]:
context_pos = 'Vị trí, chức_năng, nhiệm vụ, quyền_hạn của tổ_chức cung_cấp dịch_vụ chứng_thực chữ_ký số chuyên_dùng Chính_phủ 1. Tổ_chức cung_cấp dịch_vụ chứng_thực chữ_ký số chuyên dùng Chính phủ là tổ_chức trực_thuộc Ban Cơ_yếu Chính_phủ, cung_cấp dịch_vụ chứng_thực chữ_ký số chuyên_dùng Chính_phủ cho các cơ_quan Đảng và Nhà_nước . 2  . Tổ_chức cung_cấp dịch_vụ chứng_thực chữ_ký số chuyên_dùng Chính_phủ có nhiệm_vụ , quyền_hạn sau : a) Quản_lý , duy_trì , vận_hành hệ_thống kỹ_thuật để cung_cấp dịch_vụ chứng_thực chữ_ký số chuyên_dùng Chính_phủ cho các cơ_quan Đảng và Nhà_nước ; b) Tự_cấp chứng_thư số cho mình ; c ) Xây_dựng , trình cấp có thẩm_quyền ban_hành và tổ_chức hướng_dẫn quy_trình nghiệp_vụ về việc cung_cấp , quản_lý , sử_dụng dịch_vụ chứng_thực chữ_ký số chuyên_dùng Chính_phủ ; d ) Định_kỳ hàng năm hướng_dẫn các cơ_quan , tổ_chức , cá_nhân báo_cáo và tổ_chức tổng_kết công_tác quản_lý , triển_khai sử_dụng chứng_thư số và dịch_vụ chứng_thực chữ_ký số trong các cơ_quan Đảng , Nhà_nước ; đ) Được Nhà_nước bố_trí , đảm_bảo nhân_sự , kinh_phí và trụ_sở làm_việc để triển_khai các nhiệm_vụ , quản_lý , duy_trì hoạt_động , bảo_đảm cung_cấp chứng_thư số và dịch_vụ chứng_thực chữ_ký số theo nhu_cầu thực_tế của các cơ_quan Đảng , Nhà_nước và an_ninh , an_toàn theo quy_mô hoạt_động .'

In [66]:
idx = 11
query = test.iloc[idx]['question']
context_neg = test.iloc[idx]['negative_context'][0]
# context_pos = test.iloc[idx]['context'][0]

tokenized_pairs_pos = [query, context_pos]
tokenized_pairs_neg = [query, context_neg]

scores_pos = model_encode.predict(tokenized_pairs_pos, show_progress_bar = False)
scores_neg = model_encode.predict(tokenized_pairs_neg, show_progress_bar = False)

print(scores_pos)
print(scores_neg)

0.13149662
0.0037553585


In [32]:
context_neg

'Quản_lý , sử_dụng số tiền thu được từ bán tài_sản trên đất , chuyển_nhượng quyền sử_dụng đất của công_ty cổ_phần 1 . Tiền bán tài_sản trên đất , chuyển_nhượng quyền sử_dụng đất được quản_lý , sử_dụng như sau : a ) Đối_với tài_sản trên đất đã được xác_định là phần vốn nhà_nước tại công_ty cổ_phần theo đúng quy_định của pháp_luật , công_ty cổ_phần được sử_dụng số tiền bán tài_sản trên đất theo quy_định của pháp_luật về doanh_nghiệp và pháp_luật về quản_lý , sử_dụng vốn nhà_nước đầu_tư vào sản_xuất , kinh_doanh tại doanh_nghiệp ; b ) Đối_với giá_trị quyền sử_dụng đất đã được xác_định là phần vốn nhà_nước tại công_ty cổ_phần theo đúng quy_định của pháp_luật , công_ty cổ_phần được sử_dụng số tiền chuyển_nhượng quyền sử_dụng đất tương_ứng với mục_đích sử_dụng đất khi xác_định giá_trị phần vốn nhà_nước tại công_ty cổ_phần và thời_hạn sử_dụng đất còn lại theo quy_định của pháp_luật về doanh_nghiệp , pháp_luật về quản_lý , sử_dụng vốn nhà_nước đầu_tư vào sản_xuất , kinh_doanh tại doanh_nghiệp 

In [41]:
context_pos

'Vị_trí , chức_năng , nhiệm_vụ , quyền_hạn của tổ_chức cung_cấp dịch_vụ chứng_thực chữ_ký số chuyên_dùng Chính_phủ 1  . Tổ_chức cung_cấp dịch_vụ chứng_thực chữ_ký số chuyên_dùng Chính_phủ là tổ_chức trực_thuộc Ban Cơ_yếu Chính_phủ , cung_cấp dịch_vụ chứng_thực chữ_ký số chuyên_dùng Chính_phủ cho các cơ_quan Đảng và Nhà_nước . 2  . Tổ_chức cung_cấp dịch_vụ chứng_thực chữ_ký số chuyên_dùng Chính_phủ có nhiệm_vụ , quyền_hạn sau : a ) Quản_lý , duy_trì , vận_hành hệ_thống kỹ_thuật để cung_cấp dịch_vụ chứng_thực chữ_ký số chuyên_dùng Chính_phủ cho các cơ_quan Đảng và Nhà_nước ; b ) Tự_cấp chứng_thư số cho mình ; c ) Xây_dựng , trình cấp có thẩm_quyền ban_hành và tổ_chức hướng_dẫn quy_trình nghiệp_vụ về việc cung_cấp , quản_lý , sử_dụng dịch_vụ chứng_thực chữ_ký số chuyên_dùng Chính_phủ ; d ) Định_kỳ hàng năm hướng_dẫn các cơ_quan , tổ_chức , cá_nhân báo_cáo và tổ_chức tổng_kết công_tác quản_lý , triển_khai sử_dụng chứng_thư số và dịch_vụ chứng_thực chữ_ký số trong các cơ_quan Đảng , Nhà_nướ